# <font color='red'> Phase Diagram of a Harmonic Oscillator </font>

## 1. Using Arrays

### * The equation of motion for a harmonic oscillator is $\dot{x}=\frac{p}{m}$ and $\dot{p}=-m \omega^2 x$. Lets solve this with first order solver.

In [ ]:
from matplotlib import pyplot as plt # We need this package to plot figures.

In [ ]:
def Simple_Evolve_SHO(x,p,δt,m,ω): # Returns ẋ and ṗ for SHO.
    dx = (p/m)*δt
    dp = -m*(ω**2)*x*δt
    return dx,dp

In [ ]:
# TEST as you write...
dx,dp = Simple_Evolve_SHO(2,1,1e-4,1,1)
dx,dp

### * Do not use for Loops unless you need to. Lets see this in action by first building a "loopy way" of evaluating all this.

In [ ]:
def Evolve_Many_Times(x0,p0,δt,m,ω,num):
        x=[] # Empty arrays, so that you can store x and p. Remove this and see the error python will throw.
        p=[] # Empty arrays, so that you can store x and p. Remove this and see the error python will throw.
        x.append(x0) # Append function simply adds a given entry, x0 here, as the last entry to the list.
        p.append(p0) # APPEND IS A METHOD, which means that you use .append() method to do things. This should remind you of Object Oriented Programming...
        for i in range(1,num): # WHY did I write range(1,num) and not just range(num)?
            δx,δp = Simple_Evolve_SHO(x[-1],p[-1],δt,m,ω)
            x.append(x[-1]+δx)
            p.append(p[-1]+δp)
        return x,p

In [ ]:
x0=.0
p0=.01
x,p=Evolve_Many_Times(x0,p0,1e-5,1,1,10**6)

In [ ]:
# Notice that this is a quick way to plot x,
# since I did not give the x-axis, plt plots x on the y-axis vs. array number on the x-axis.
plt.plot(x,'k')

In [ ]:
plt.plot(x,p,'b+')
#plt.axes().set_aspect('equal', 'datalim')

In [ ]:
for i in range(6):
        %timeit Evolve_Many_Times(x0,p0,1e-5,1,1,10**i)

In [ ]:
#IF you plot it, there is a bunch of points at the origin...
plt.plot([1,10,100,1000,10000,100000],[615*1e-9,8.06*1e-6,81.6*1e-6,818*1e-6,8.3*1e-3,85.3*1e-3],'rx-')

In [ ]:
# Use semilogy or loglog to visualize this better
plt.loglog([1,10,100,1000,10000,100000],[615*1e-9,8.06*1e-6,81.6*1e-6,818*1e-6,8.3*1e-3,85.3*1e-3],'rd--')

## We learnt from the basic numpy notebook that numpy typically is better at this stuff. So lets see if we can make things better?

# 3. Using Integrators 
#### (see https://flothesof.github.io/harmonic-oscillator-three-methods-solution.html)

In [ ]:
import numpy as np

# In integrators , typically you are trying to solve a 1st order vector ODE of the form $\dot{\vec{y}}=f(\vec{y})$. You need to define the RHS.

In [ ]:
def deriv(u, t, omega_squared):
    "Provides derivative of vector u."
    xdot, x = u
    return [-omega_squared * x, xdot]

In [ ]:
from scipy.integrate import odeint
y0 = [0, 1]
snapshot_dt = 0.3
ts = np.arange(0, 12, snapshot_dt)
scipysol = odeint(deriv, y0, ts, args=(1,))

In [ ]:
odeint?

In [ ]:
scipysol

In [ ]:
plt.plot(ts, scipysol[:, 1])

# 4. Doing things Smartly. Exponential of the evolution operator.

In [ ]:
import numpy as np
from scipy.linalg import expm # This is the "exponential of an operator" function

In [ ]:
expm?

In [ ]:
expm(np.zeros((2,2)))

# Notice that you can write $\vec{z}(t)=e^{Mt}\vec{z}(0)$

In [ ]:
def Integrate_Using_Exp(δt,m,ω):
        M=np.array([[0,1/m],[-m*(ω**2),0]]) # Evaluate the matrix
        Evolution=expm(M*δt) # Evaluate its exponential
        return Evolution # Return it.

In [ ]:
z0=np.random.rand(2,1) # Initialize the phase trajectory somewhere random in the unit circle.
z0

In [ ]:
U=Integrate_Using_Exp(1e-5,1,1)
U

In [ ]:
def Evolve_Using_Numpy(U,z0,num):
    x=np.zeros(num)
    p=np.zeros(num)
    z=z0
    for i in range(num):
        x[i]=z[0]
        p[i]=z[1]
        z = U.dot(z)
        
    return(x,p)

In [ ]:
Evolve_Using_Numpy(U,z0,10**5)

In [ ]:
plt.plot(x,p,'b.')
#plt.axes().set_aspect('equal', 'datalim')

In [ ]:
for i in range(6):
        %timeit Evolve_Using_Numpy(U,z0,10**i)

# Comparing with before, nothing good seems to have been achieved. 
# Reason for that is we are not using vectorization properly. 
# We are still going through loops.

In [ ]:
for i in range(6):
        %timeit Evolve_Many_Times(x0,p0,1e-5,1,1,10**i)

In [ ]:
z0

In [ ]:
U

In [ ]:
from numpy.linalg import matrix_power
matrix_power(U, 2).dot(z0).transpose()

In [ ]:
def Evolve_Using_Numpy_List_Comp(U,z0,num): # I have added List_Comp for list comprehension
    Z=[(matrix_power(U, i)).dot(z0).transpose() for i in range(num)]
    return Z

In [ ]:
Z=Evolve_Using_Numpy_List_Comp(U,z0,5)

In [ ]:
for i in range(5):
    %timeit Evolve_Using_Numpy_List_Comp(U,z0,10**i)

## Still no speedup. So whats happening?


In [ ]:
Dim = 10


In [ ]:
δt=1e-5
F = np.random.rand(Dim,Dim) +1j*np.random.rand(Dim,Dim) 
Fd=F.conj().T
H=F.dot(Fd)
U=expm(-1j*H*δt)

In [ ]:
z0=np.random.rand(Dim)

In [ ]:
U.dot(z0)

In [ ]:
for i in range(5):
    %timeit Evolve_Using_Numpy_List_Comp(U,z0,10**i)

# EXERCISES: 
## Write a code to solve time dependant phase trajectory for damped SHO $\ddot{x}+\omega^2 x+\dot{x}=\epsilon\cos(\omega_d t)$

In [ ]:
import numpy as np

In [ ]:
z0=np.array([1,0])

In [ ]:
θ=np.pi/3
M=np.array([[np.cos(θ),np.sin(θ)],[-np.sin(θ),np.cos(θ)]]) # Rotation matrix. 

In [ ]:
def Repeat_Mul(M,z0,n):
    if n==0:
        return z0
    else:
        return M@Repeat_Mul(M,z0,n-1)

In [ ]:
for i in range(4):
    %timeit Repeat_Mul(M,z0,10**i) # Memory issues

In [ ]:
def Repeat_Mul_2(M,z0,n):
    if n==1:
        return z0
    else:
        for i in range(1,n+1):
            z0 = M@z0
        return z0    

In [ ]:
for i in range(5):
    %timeit Repeat_Mul_2(M,z0,10**i) # Memory issues

In [ ]:
Repeat_Mul_2(M,z0,136)

# Using "List Comprehension" for time dependant Hamiltonians.

In [ ]:
t=np.linspace(0,10,100)
ω=np.pi/5

In [ ]:
σx=np.array([[0,1],[1,0]])
σz=np.array([[1,0],[0,-1]])

In [ ]:
H=[σz+np.cos(ω*t0)*σx for t0 in t]

In [ ]:
H[54]